In [ ]:
import numpy as np
#data = np.delete(np.genfromtxt('Corners(F).csv',delimiter=','),obj=0,axis=0)
data = np.genfromtxt('Vowel.csv')
print(data.shape)

In [ ]:
import time
time1 = time.time()

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
data = np.delete(np.genfromtxt('optdigits_csv.csv',delimiter=','),obj=0,axis=0)
print(len(data),"\t", data.shape[1])

from sklearn.cluster import KMeans

distortions = []
K = range(1,15)
#K = range(1,30)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(data)
    distortions.append(kmeanModel.inertia_)
plt.figure()
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

from sklearn.metrics import silhouette_score

sil = []
#kmax = 10
kmax = 15

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
    kmeans = KMeans(n_clusters = k).fit(data)
    labels = kmeans.labels_
    sil.append(silhouette_score(data, labels, metric = 'euclidean'))

plt.figure()
plt.plot(K, sil, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('The Silhouette Score Method showing the optimal k')
plt.show()

elapsed1 = time.time() - time1
print(elapsed1, " sec")

In [ ]:
# Weighted K-means Algorithm
    
from __future__ import division, print_function

import random
import sklearn.datasets

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from colorama import Style


def euclidean(a, b):
    return np.linalg.norm(np.asarray(a) - np.asarray(b))


class WKMeans():
    def counted(f):
        def wrapped(*args, **kwargs):
            wrapped.calls += 1
            return f(*args, **kwargs)
        wrapped.calls = 0
        return wrapped

    def __init__(self, K, X=None, N=0, c=None, alpha=0, beta=0, dist=euclidean,
                 max_runs=200, label='My Clustering', verbose=True, mu=None,
                 max_diff=0.001):
        self.K = K
        if X is None:
            if N == 0:
                raise Exception("If no data is provided, \
                                 a parameter N (number of points) is needed")
            else:
                self.N = N
                self.X = self._init_gauss(N)
        else:
            self.X = X
            self.N = len(X)
        self.mu = mu
        self.old_mu = None
        if self.mu is None:
            self.method = 'random'
        else:
            self.method = 'manual'
        self.clusters = None
        self.cluster_indices = np.asarray([None for i in self.X])
        self.alpha = alpha
        self.scaling_factor = np.ones((self.K)) / self.K
        self.beta = beta
        if c is None:
            self.counts_per_data_point = [1 for x in self.X]
        else:
            self.counts_per_data_point = c
        self.counts_per_cluster = [0 for x in range(self.K)]
        self.max_runs = max_runs
        self.runs = 0
        self.dist = dist
        self.max_diff = max_diff
        self.label = label
        self.verbose = verbose

    def _init_gauss(self, N):
        centers = [[0, 0], [1, 0], [0.5, np.sqrt(0.75)]]
        cluster_std = [0.3, 0.3, 0.3]
        n_samples = int(np.ceil(0.75 * N))

        data, labels_true = \
            sklearn.datasets.make_blobs(n_samples=n_samples,
                                        centers=centers,
                                        cluster_std=cluster_std)

        centers = [[0.5, np.sqrt(0.75)]]
        cluster_std = [0.3]
        # n_clusters = len(centers)
        extra, labels_true = \
            sklearn.datasets.make_blobs(n_samples=int(0.25 * N),
                                        centers=centers,
                                        cluster_std=cluster_std)

        data = np.concatenate((data, extra), axis=0)
        return data

    @counted
    def plot_clusters(self, snapshot=0):
        X = self.X
        # fig = plt.figure(figsize=(5, 5))
        # ax = plt.gca()
        # palette = itertools.cycle(sns.color_palette())

        if self.mu and self.clusters:
            mu = self.mu
            clus = self.clusters
            K = self.K
            for m, clu in enumerate(clus):
                cs = cm.spectral(1. * m / K)
                plt.plot(list(zip(*clus[m]))[0], list(zip(*clus[m]))[1], '.',
                         markersize=8, color=cs, alpha=0.5)
                plt.plot(mu[m][0], mu[m][1], 'o', marker='*',
                         markersize=12, color=cs,  markeredgecolor='white',
                         markeredgewidth=1.0)
        else:
            plt.plot(list(zip(*X))[0], list(zip(*X))[1], '.', alpha=0.5)

        if self.method == '++':
            title = 'K-means++'
        elif self.method == 'random':
            title = 'K-means with random initialization'
        elif self.method == 'seed':
            title = 'K-means seeded with pre-loaded clusters'
        pars = r'$N=%s, K=%s, \alpha=%s$' % (str(self.N), str(self.K),
                                             str(self.alpha))
        plt.title('\n'.join([pars, title]), fontsize=16)

        plt.savefig('kpp_N_%i_K_%i_alpha_%i_beta_%i_%i.png' % (self.N, self.K,
                                                               self.alpha,
                                                               self.beta,
                                                               snapshot),
                    bbox_inches='tight',
                    dpi=200)

    @counted
    def _cluster_points(self):
        clusters = [[] for i in range(self.K)]
        counts_per_cluster = [0 for i in range(self.K)]

        for index, x in enumerate(self.X):
            bestmukey = min([(i[0],
                              self.scaling_factor[i[0]] *
                              self.dist(x, self.mu[i[0]]))
                             for i in enumerate(self.mu)],
                            key=lambda t: t[1])[0]
            clusters[bestmukey].append(x)
            counts_per_cluster[bestmukey] += self.counts_per_data_point[index]
            self.cluster_indices[index] = bestmukey

        clusters = [c for c in clusters if len(c)]
        scaling_factor = np.asarray([self.scaling_factor[i] for i in range(self.K) if counts_per_cluster[i]])
        counts_per_cluster = [num for num in counts_per_cluster if num]  
        self.clusters = clusters
        self.scaling_factor = scaling_factor
        self.counts_per_cluster = counts_per_cluster
        self.K = len(self.clusters)

        if self.verbose:
            print('\tNumber of unique items per cluster: ' + Style.BRIGHT,
                  end='')
            print([len(x) for x in self.clusters], end='')
            print(Style.RESET_ALL)
            print('\tNumber of items per cluster: ' + Style.BRIGHT,  end='')
            for i, c in enumerate(self.counts_per_cluster):
                if i == 0:
                    print('[', end='')
                print('%1.1f' % c, end='')
                if i == len(self.counts_per_cluster) - 1:
                    print(']', end='')
                else:
                    print(', ', end='')
            print(Style.RESET_ALL)

        scaling_factor = np.asarray([self.counts_per_cluster[index]**self.alpha
                                     for index, cluster in
                                     enumerate(self.clusters)])

        scaling_factor = scaling_factor / np.sum(scaling_factor)

        scaling_factor = (1 - self.beta) * scaling_factor + (self.beta) * self.scaling_factor

        self.scaling_factor = scaling_factor

        if self.verbose:
            print('\tScaling factors per cluster: ' + Style.BRIGHT,  end='')
            for i, c in enumerate(self.scaling_factor):
                if i == 0:
                    print('[', end='')
                print('%1.3f' % c, end='')
                if i == len(self.scaling_factor) - 1:
                    print(']', end='')
                else:
                    print(', ', end='')
            print(Style.RESET_ALL)

    def _reevaluate_centers(self):
        new_mu = []
        for k in self.clusters:
            new_mu.append(np.mean(k, axis=0))

        self.mu = new_mu

    def _has_converged(self):
        diff = 1000
        if self.clusters:            
            for clu in self.clusters:
                if len(clu) is 0:
                    raise ValueError('One or more clusters disappeared because'
                                     ' all points rushed away to other'
                                     ' cluster(s). Try increasing the'
                                     ' stickiness parameter (beta).')
            diff = 0
            for i in range(self.K):
                diff += self.dist(self.mu[i].tolist(), self.old_mu[i].tolist())
            diff /= self.K

            if self.verbose:
                print('\tDistance between previous and current centroids: ' +
                      Style.BRIGHT + str(diff) + Style.RESET_ALL)

        return diff < self.max_diff

    def find_centers(self, method='random'):
        self.method = method
        X = self.X
        K = self.K
        self.old_mu = random.sample(list(X), K)

        if method == 'random':
            self.mu = random.sample(X, K)

        while not self._has_converged() and self.runs < self.max_runs:
            if self.verbose:
                print(Style.BRIGHT + '\nRun: ' + str(self.runs) + ', alpha: ' +
                      str(self.alpha) + ', beta: ' +
                      str(self.beta) + ', label: '
                      + self.label + Style.RESET_ALL)
            self.old_mu = self.mu
            self._cluster_points()
            self._reevaluate_centers()
            self.runs += 1

        
        if self.verbose:
            print(Style.BRIGHT + '\nThe End!' + Style.RESET_ALL)
            print('\tLabel: ' + Style.BRIGHT + self.label + Style.RESET_ALL)
            print('\tTotal runs:' + Style.BRIGHT, self.runs, Style.RESET_ALL)
            print('\tNumber of unique items per cluster: ' + Style.BRIGHT, end='')
            print([len(x) for x in self.clusters], end='')
            print(Style.RESET_ALL)
            print('\tNumber of items per cluster: ' + Style.BRIGHT,  end='')
            for i, c in enumerate(self.counts_per_cluster):
                if i == 0:
                    print('[', end='')
                print('%1.1f' % c, end='')
                if i == len(self.counts_per_cluster) - 1:
                    print(']', end='')
                else:
                    print(', ', end='')
            print(Style.RESET_ALL)


class KPlusPlus(WKMeans):
    
    def _dist_from_centers(self):
        cent = self.mu
        X = self.X
        D2 = np.array([min([self.dist(x, c)**2 for c in cent]) for x in X])
        self.D2 = D2

    def _choose_next_center(self):
        self.probs = self.D2 / self.D2.sum()
        self.cumprobs = self.probs.cumsum()
        r = random.random()
        ind = np.where(self.cumprobs >= r)[0][0]
        return(self.X[ind])

    def init_centers(self):
        self.mu = random.sample(list(self.X), 1)
        while len(self.mu) < self.K:
            self._dist_from_centers()
            self.mu.append(self._choose_next_center())

    def plot_init_centers(self):
        X = self.X
        # fig = plt.figure(figsize=(5, 5))
        plt.xlim(-1, 1)
        plt.ylim(-1, 1)
        plt.plot(list(zip(*X))[0], list(zip(*X))[1], '.', alpha=0.5)
        plt.plot(list(zip(*self.mu))[0], list(zip(*self.mu))[1], 'ro')
        plt.savefig('kpp_init_N%s_K%s.png' % (str(self.N), str(self.K)),
                    bbox_inches='tight', dpi=200)


###############################################################################################################################
###############################################################################################################################


# Str Kmeans algorithm

import numpy as np 
from numpy import linalg as LA
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import random

def StrmKMmeans(x, Centroids, Assign, cof_f, max_Cen, beta, DataType = 'Otherwise'):
    m = len(x)

    if not Centroids.size: 
        #print "Entered the first time!"
        x = np.append(x, x)
        x = np.append(x, 1)
        Centroids = x
        Centroids = x.reshape(1, len(x))
        Assign = np.append(Assign, 0)

    else: 
        Dist = np.sum((Centroids[:,:m] - x)**2, axis = 1)
        delta_y = np.min(Dist)
        idx_y = np.argmin(Dist)

        if random()<delta_y/cof_f: # Create new facility for x
            x = np.append(x, x)
            x = np.append(x, 1)
            Centroids = np.append( Centroids, [x], axis = 0 )
            Assign = np.append(Assign, Centroids.shape[0]-1)
        else: # Add x to an existing facility
            Centroids[idx_y, m:2*m] = Centroids[idx_y, m:2*m] + x
            Centroids[idx_y, 2*m] += 1
            Assign =  np.append(Assign, idx_y)
            
    while Centroids.shape[0]>max_Cen: 
        cof_f = beta * cof_f;

        for i in range(Centroids.shape[0]):
            if DataType == 'Text':
                Centroids[i, :m] = Centroids[i, m:2*m]/Centroids[i, 2*m]
                Centroids[i, :m] = Centroids[i, m:2*m]/LA.norm(Centroids[i, m:2*m])
            else:
                Centroids[i, :m] = Centroids[i, m:2*m]/Centroids[i, 2*m]
        
        Centroids_til = Centroids[0, :]
        Centroids_til = Centroids_til.reshape(1, len(Centroids_til))

        for i in range(1, Centroids.shape[0]): # for each z in the preivous facility C
            
            Dist_til = np.sum((Centroids_til[:,:m] - Centroids[i, :m])**2, axis = 1)
            delta_z = np.min(Dist_til)
            idx_z = np.argmin(Dist_til)

            if random()<(delta_z*Centroids[i, 2*m]/cof_f):
                Centroids_til = np.append(Centroids_til, [Centroids[i, :]], axis = 0)
            else:
                Assign[Assign[:]==i] = idx_z
                Centroids_til[idx_z, m:2*m] = Centroids_til[idx_z, m:2*m] + Centroids[i, m:2*m]
                Centroids_til[idx_z, 2*m] += Centroids[i, 2*m]
                
        Centroids = Centroids_til

    return Centroids, Assign, cof_f


"""
x = np.array([[0.5,0.5], [0.3,0.2], [-1,-1], [2,1], [1,-1], [2,2], [3,5]])
Centroids = np.array([])
Assign = []
for i in range(0, x.shape[0]):
    print "Step", i, ": current point is ", x[i]
    [Centroids, Assign, cof_f] = StrmKMmeans(x[i], Centroids, Assign, 10.0, 2, 5, DataType = 'Text')
    print "Step", i, "finished!"
    print Centroids, Assign
"""


###############################################################################################################################
###############################################################################################################################


# MiLOF Algorithm

# Inputs:
    # kpar: number of nearest neighbours
    # dimension: dimension of data points
    # buck: bucket size (memory limit)
    # filepath: path of input data file
    # num_k: number of clusters used in kmeans, streaming kmeans and weighted kmeans
    # width: N/A

import time
time2 = time.time()
    
    
import numpy as np
import scipy.io as sio
import configparser
#import wKmeans as wkm
#import strmKMeans as skm
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt


Scores = []

class Point:
    def __init__(self):
        self.kdist = []
        self.knn = []
        self.lrd = []
        self.LOF = []
        # self.rdist = {}                                                                 ##Can I turn this off later?

class Cluster:
    def __init__(self):
        self.center = []
        # self.LS = []                                                                    ##Can I turn this off later?

def LOF(datastream, kpar):
    Points = Point()
    clf = LocalOutlierFactor(n_neighbors=kpar, algorithm="kd_tree", leaf_size=30, metric='euclidean',contamination='auto')  
    #Changed algorithm from kd_tree to auto; Changed metric from euclidean to minkowski with p=2;
    clf.fit(datastream)
    Points.LOF = [-x for x in clf.negative_outlier_factor_.tolist()]  #Gets the LOF score of each element
    Points.lrd = clf._lrd.tolist()                                    #Gets the lrd score of each element
    dist, ind = clf.kneighbors()                               #Gets the dist of an element from its k NN's and their indexes
    Points.kdist = dist.tolist()                                      #Converts the dist array and stores it as a list
    Points.knn = ind.tolist()                                         #Converts the index array and stores it as a list
    return Points

def ComputeDist(A, B):                        #Computes the linear distance between two arrays A and B after normalizing them  
    dist = np.linalg.norm(A-B)     
    return dist

def union(list1, list2):                                                                  ##Why is this function used?
    set1 = set(list1)
    set2 = set(list2)
    list3 = list1 + list(set2 - set1)
    return list3

def setdiff(list1, list2):                                                                ##Why is this function used?     
    set1 = set(list1)
    set2 = set(list2)
    list3 = list(set1 - set2)
    return list3

def IncrementalLOF_Fixed(Points, datastream, PointsC, Clusters, kpar, buck, width):
    i = datastream.shape[0]
    # print("******************* Processing Data Point", i-1, "*******************")
    
    nbrs = NearestNeighbors(n_neighbors=kpar, algorithm='brute', metric='euclidean')
    #Changed algorithm from brute to auto; Changed metric from euclidean to minkowski with p=2;
    nbrs.fit(datastream[0:i-1, :])                                 #Fitting the NN on the dataset
    dist, ind = nbrs.kneighbors(datastream[i-1, :].reshape(1, -1)) #Getting the dist and index of the n neighbors
    Points.kdist = Points.kdist + dist.tolist()                                           ##Why doing this?
    Points.knn = Points.knn + ind.tolist()                                                ##Why doing this?
    # print("Points.kdist = ", Points.kdist)
    # print("Points.knn = ", Points.knn)

    dist = []
    for j in range(0, len(Clusters.center)):
        distCI = ComputeDist(Clusters.center[j], datastream[i-1, :]) - width;
        # if distCI <=0:
        # 	distCI=PointsC.kdist[j]
        dist = dist + [distCI]
    # print("dist = ", dist)

    if len(dist):
        minval, ind =  min((dist[j], j) for j in range(0, len(dist)))
        for j in range(0, len(Points.kdist[i-1])):
            if minval < Points.kdist[i-1][j]:
                Points.kdist[i-1][j] = minval
                Points.knn[i-1][j] = buck + ind
                if j < len(Points.kdist[i-1])-1:
                    Points.kdist[i-1][j+1:] = [minval] * (len(Points.kdist[i-1]) - j - 1)
                    Points.knn[i-1][j+1:] = [buck+ind] * (len(Points.kdist[i-1]) - j - 1)
                break

    rNN = []
    for k in range(0, i-1):
        distance = ComputeDist(datastream[k, :], datastream[i-1, :])
        # print ("distance between point", k, "and point", i-1, "=", distance)
        if (Points.kdist[k][-1] >= distance):
            for kk in range(0, len(Points.knn[k])):
                if distance <= Points.kdist[k][kk]:
                    if kk == 0:
                        Points.knn[k]   = [i-1] + Points.knn[k][kk:]
                        Points.kdist[k] = [distance] + Points.kdist[k][kk:]
                    else:
                        Points.knn[k]   = Points.knn[k][0:kk] + [i-1] + Points.knn[k][kk:]
                        Points.kdist[k] = Points.kdist[k][0:kk]+ [distance] + Points.kdist[k][kk:]
                    break

            for kk in range(kpar, len(Points.knn[k])):
                # print(Points.kdist[k][kk], Points.kdist[k][kpar-1])
                if Points.kdist[k][kk] != Points.kdist[k][kpar-1]:
                    del Points.kdist[k][kk:]
                    del Points.knn[k][kk:] 
                    break

            rNN = rNN + [k]
    # print("rNN = ", rNN)
    # print("Points.kdist = ", Points.kdist)
    # print("Points.knn = ", Points.knn)

    updatelrd = rNN
    if len(rNN) > 0:
        for j in rNN:
            for ii in Points.knn[j]:
                if ii < len(Points.knn) and ii != i-1 and j in Points.knn[ii]:
                    updatelrd = union(updatelrd, [ii])
    # print("updatelrd = ", updatelrd)

    rdist = 0
    for p in Points.knn[i-1]:
        if p > buck-1:
            rdist = rdist + max(ComputeDist(datastream[i-1, :], Clusters.center[p-buck]) - width, PointsC.kdist[p-buck])
        else:
            rdist = rdist + max(ComputeDist(datastream[i-1, :], datastream[p,:]), Points.kdist[p][-1])
    Points.lrd = Points.lrd + [1/(rdist/len(Points.knn[i-1]))]
    # print("Points.lrd = ", Points.lrd)

    updateLOF = updatelrd
    if len(updatelrd) > 0:
        for m in updatelrd:
            rdist = 0
            for p in Points.knn[m]:
                if p > buck-1:
                    rdist = rdist + max(ComputeDist(datastream[m, :], Clusters.center[p-buck])-width, PointsC.kdist[p-buck])
                else:
                    rdist = rdist + max(ComputeDist(datastream[m, :], datastream[p, :]), Points.kdist[p][-1])
            Points.lrd[m] = 1/(rdist/len(Points.knn[m]))
            for k in range(0, len(Points.knn)):
                if k != m and Points.knn[k].count(m) > 0:
                    updateLOF = union(updateLOF, [k])
    # print("Points.lrd =", Points.lrd)
    # print("updateLOF =", updateLOF)

    # LOF neighbours
    if len(updateLOF) > 0:
        for l in updateLOF:
            lof = 0
            for ll in Points.knn[l]:
                if ll > buck-1:
                    lof = lof + PointsC.lrd[ll-buck]/Points.lrd[l]
                else:
                    lof = lof + Points.lrd[ll]/Points.lrd[l]
            if l == len(Points.LOF):
                Points.LOF = Points.LOF + [lof/len(Points.knn[l])]
            else:
                Points.LOF.append(lof/len(Points.knn[l]))
                #Points.LOF[l] = lof/len(Points.knn[l])
    # print("Points.LOF =", Points.LOF)

    # LOF-i
    lof = 0
    for ll in Points.knn[i-1]:
        if ll > buck-1:
            lof = lof + PointsC.lrd[ll-buck]/Points.lrd[i-1]
        else:
            lof = lof + Points.lrd[ll]/Points.lrd[i-1]
    if i-1 == len(Points.LOF):
        Points.LOF = Points.LOF + [lof/len(Points.knn[i-1])]
    else:
        Points.LOF[i-1] = lof/len(Points.knn[i-1])

    # print("Points.knn =", Points.knn)
    # print("Points.LOF =", Points.LOF)
    # print("Points.lrd =", Points.lrd)
    # print("rNN = ", rNN)
    # print("updatelrd = ", updatelrd)
    # print("updateLOF =", updateLOF)
    
    return Points

def MILOF(kpar,dimension,buck,filename,num_k,width):
    #config = configparser.ConfigParser()
    #config.read(configFile)
    #filepath = config['Analyzer']['InputMatFile']
    filename = filename
    #dimension = int(config['Analyzer']['Dimension'])
    dimension = dimension
    #num_k = int(config['Analyzer']['NumK'])
    num_k = num_k
    #kpar = int(config['Analyzer']['KPar'])
    kpar = kpar
    #buck = int(config['Analyzer']['Bucket'])
    buck = buck
    #width = int(config['Analyzer']['Width'])
    width = width
    
    #datastream = sio.loadmat(filepath)                                             #Commented it
    #datastream = np.array(datastream['DataStream'])                                #Commented it
    # datastream = datastream[0:10*buck, :]
    #datastream = np.delete(np.genfromtxt(filename,delimiter=','),obj=0,axis=0)
    datastream = np.genfromtxt(filename,delimiter=',')                              #Can use this when comma is absent
    datastream_for_plotting = datastream
    #datastream = datastream[:, 0:dimension]                                        #Commented it
    #datastream = np.unique(datastream, axis=0)                                     #Commented it

    scaler = MinMaxScaler()
    scaler.fit(datastream)
    datastream = scaler.transform(datastream)
    PointNum = datastream.shape[0]
    print ("number of data points =", PointNum)
    # print ("normalized data = ", datastream)

    hbuck = int(buck/2) # half of buck
    kdist = []
    Scores = []
    clusterLog = []
    Clusters = Cluster()
    PointsC= Point()
    Points = LOF(datastream[0:kpar+1, :], kpar)
    Scores = Scores + Points.LOF

    for i in range(0, kpar+1):
        kdist = kdist + [Points.kdist[i][-1]]

    #print("Scores =", Scores)    ## I have turned it off
    #print(len(Scores))           ## I have turned it off
    #print("kdist =", kdist)

    for i in range(kpar+2, hbuck+1):
        Points = IncrementalLOF_Fixed(Points, datastream[0:i, :], PointsC, Clusters, kpar, buck, width)
        Scores = Scores + [Points.LOF[i-1]]
        kdist  = kdist + [Points.kdist[i-1][-1]]

    # print("Points.kdist =", Points.kdist)
    # print("Points.knn =", Points.knn)
    # print("Points.LOF =", Points.LOF)
    # print("Points.lrd =", Points.lrd)
    # print("Scores =", Scores)	
    # print("kdist =", kdist)

    exit = False
    step = 0
    while not exit:
        for i in range(hbuck+1, buck+1):
            if i > datastream.shape[0]:
                exit = True
                break
            Points = IncrementalLOF_Fixed(Points, datastream[0:i, :], PointsC, Clusters, kpar, buck, width)
            Scores = Scores + [Points.LOF[i-1]]
            kdist  = kdist + [Points.kdist[i-1][-1]]

        # print("Scores =", Scores)
        # print("kdist =", kdist)
        # print("Points.kdist =", Points.kdist)
        # print("Points.knn =", Points.knn)
        # print("Points.LOF =", Points.LOF)
        # print("Points.lrd =", Points.lrd)

        if not exit:
            step = step + 1

            print("*******************Step", step, ": processing data points", step*hbuck, "to", (step+1)*hbuck, "*******************")

            indexNormal = list(range(0, hbuck))
            kmeans = KMeans(n_clusters=num_k, init='k-means++', max_iter=100, n_jobs=-1)  # Considering precompute_distances for faster but more memory
            kmeans.fit(datastream[0:hbuck, :]) # need to check how to configure to match result of matlab code 
            center = kmeans.cluster_centers_
            clusterindex = kmeans.labels_

            # print("label =", clusterindex)
            # print("center =", center)

            remClustLbl = list(range(0, num_k))
            lof_scores = []
            for itr in range(0, hbuck):
                lof_scores = lof_scores + [Points.kdist[itr][-1]]
            lof_scores = np.array(lof_scores)
            lof_threshold = np.mean(lof_scores) + 3 * np.std(lof_scores) # Not sure if calcuating for each i is necessary

            # print("lof_scores=", lof_scores)
            # print("lof_threshold=", lof_threshold)

            for kk in range(0, num_k):
                clusterMembers = np.where(clusterindex==kk)
                clusterMembersList = np.asarray(clusterMembers).flatten().tolist()
                if np.sum(lof_scores[clusterMembers]>lof_threshold) > 0.5*len(clusterMembersList):
                    indexNormal = setdiff(indexNormal, clusterMembersList)
                    remClustLbl = setdiff(remClustLbl, [kk])
            clusterindex = clusterindex[indexNormal]
            center = center[remClustLbl,:]
            
            #print("center = ", center)
            #print("label = ", clusterindex)
        
        
            for j in range(0, len(remClustLbl)):
                num = np.sum(clusterindex == remClustLbl[j])
                PointsC.knn = PointsC.knn + [num]
                Ckdist = Clrd = CLOF = 0
                for k in np.array(indexNormal)[np.where(clusterindex==remClustLbl[j])]:
                    Ckdist = Ckdist + Points.kdist[k][-1]
                    Clrd   = Clrd   + Points.lrd[k]
                    CLOF   = CLOF   + Points.LOF[k]

                PointsC.kdist = PointsC.kdist + [Ckdist/num]
                PointsC.lrd   = PointsC.lrd   + [Clrd/num]
                PointsC.LOF   = PointsC.LOF   + [CLOF/num]

            # print(PointsC.kdist)
            # print(PointsC.lrd)
            # print(PointsC.LOF)

            datastream = np.delete(datastream, range(0,hbuck), axis=0)
            del Points.kdist[0:hbuck]
            del Points.knn[0:hbuck]
            del Points.lrd[0:hbuck]
            del Points.LOF[0:hbuck]

            # print ("remaining data = ", datastream)

            initialClusters = len(Clusters.center)
            if initialClusters > 0:
                old_center = np.array(Clusters.center)
                cluster_num = max(old_center.shape[0], center.shape[0])
                print(cluster_num)
                initial_center = []
                if cluster_num == center.shape[0]:
                    for x in center.tolist():
                        initial_center.append(np.array(x))
                else:
                    for x in old_center.tolist():
                        initial_center.append(np.array(x))

                # print("old_center = ", old_center)
                # print("center = ", center)
                # print("weights = ", PointsC.knn[0:old_center.shape[0]+center.shape[0]])

                #wkmeans = wkm.KPlusPlus(cluster_num, X=np.concatenate((old_center, center), axis=0), c=PointsC.knn[0:old_center.shape[0]+center.shape[0]], max_runs=5, verbose=False, mu=initial_center)
                #Original
                wkmeans = KPlusPlus(cluster_num, X=np.concatenate((old_center, center), axis=0), c=PointsC.knn[0:old_center.shape[0]+center.shape[0]], max_runs=5, verbose=False, mu=initial_center)
                #My
                wkmeans.find_centers(method='++')
                # strmkmeans = skm.StrmKMmeans(x=np.concatenate((old_center, center), axis=0), Centroids=initial_center, Assign=PointsC.knn[0:old_center.shape[0]+center.shape[0]], )

                merge_center = np.array(wkmeans.mu)
                mergedindex = wkmeans.cluster_indices
                cluster_num = len(merge_center)
                clusterLog = clusterLog + [cluster_num]

                PC = Point()
                for j in range(0, cluster_num):
                    num = np.sum(mergedindex==j)
                    PCknn = PCkdist = PClrd = PCLOF = 0
                    for k in np.asarray(np.where(mergedindex==j)).flatten().tolist():
                        PCknn   = PCknn   + PointsC.knn[k]
                        PCkdist = PCkdist + PointsC.knn[k] * PointsC.kdist[k]
                        PClrd   = PClrd   + PointsC.knn[k] * PointsC.lrd[k]
                        PCLOF   = PCLOF   + PointsC.knn[k] * PointsC.LOF[k]
                    
                    PC.knn   = PC.knn + [PCknn]
                    PC.kdist = PC.knn + [PCkdist / PCknn]
                    PC.lrd   = PC.knn + [PClrd / PCknn]
                    PC.LOF   = PC.knn + [PCLOF / PCknn]
                PointsC = PC
                
                Clusters.center = merge_center.tolist()
            else:
                Clusters.center = center.tolist()

            #print ("Clusters.center = ", len(Clusters.center))  #Uncommented it

            for j in range(0, len(Points.knn)):
                for k in range(0, len(Points.knn[j])):
                    if Points.knn[j][k] >= hbuck:
                        if Points.knn[j][k] < buck:
                            Points.knn[j][k] = Points.knn[j][k] - hbuck
                        else:
                            Points.knn[j][k] = mergedindex[Points.knn[j][k]-buck] + buck
                    else:
                        indarr = np.where(np.array(indexNormal)==Points.knn[j][k])
                        ind = np.asarray(indarr).flatten().tolist()
                        if len(ind):
                            cLabel = clusterindex[indarr]
                            ci = np.asarray(np.where(np.array(remClustLbl)==cLabel)).flatten().tolist()[0]
                            if not initialClusters:
                                Points.knn[j][k] = ci + buck
                            else:
                                Points.knn[j][k] = mergedindex[ci+initialClusters] + buck
                        else:
                            mindist = []
                            for kk in range(0, len(Clusters.center)):
                                mindist = mindist + [ComputeDist(datastream[j,:], Clusters.center[kk])]
                            mindistVal, ci =  min((mindist[x], x) for x in range(0, len(mindist)))
                            Points.knn[j][k] = ci + buck

            # print("Points.knn = ", Points.knn)
    #print("Scores =", Scores)
    #print(len(Scores))
    #Detect = LocalOutlierFactor.fit_predict(Scores)
    #print("place2",len(Clusters.center))  #Uncomment it
    
    
    outlier_count = 0
    outliers_for_display = []
    for element in Scores:
        if element > 3:
            outlier_count = outlier_count + 1
    #print('\nNumber of Outliers For LOF_Thresh > 2 =\t',outlier_count)
    outlier_count1 = 0
    for element1 in Scores:
        if element1 > 1.5:
            outlier_count1 = outlier_count1 + 1
    print('\nNumber of Outliers For LOF_Thresh > 1.5 =',outlier_count1)
    print('\nNumber of Outliers For LOF_Thresh > 2 =',outlier_count)
    
    
    for i in range(len(datastream_for_plotting)):
        if Scores[i] > 2:
            outliers_for_display.append(datastream_for_plotting[i])
    outliers_for_display_array = np.array(outliers_for_display)
    
    
    if dimension == 2:
        plt.figure(figsize=(8,8))
        plt.scatter(datastream_for_plotting[:,0], datastream_for_plotting[:,1], c='blue', label='Datapoints')
        plt.scatter(outliers_for_display_array[:,0], outliers_for_display_array[:,1], c='red', label='Outliers')
        plt.xlabel('X-Coordinate')
        plt.ylabel('Y-Coordinate')
        plt.title('Plotting')
        plt.legend()
        plt.show()
        
    

kpar = int(input("\nEnter the number of nearest neighbours\n"))
dimension = int(input("\nEnter the dimension of data points\n"))
buck = int(input("\nEnter the memory limit\n"))
filename = 'Vowel.csv'#str(input('\nEnter the filename\n')) #'Corners(F).csv'
num_k = int(input("\nEnter the number of clusters used in kmeans and weighted kmeans\n"))
width = int(input("\nEnter width\n"))

MILOF(kpar,dimension,buck,filename,num_k,width)

elapsed2 = time.time() - time2



print(elapsed2, " sec")